In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
%cd /gdrive/Shareddrives/neurolusion/PhD/Secondary_Structure_Pred/protpred_2d
!ls
!wget -r http://www.princeton.edu/~jzthree/datasets/ICML2014/
%cp -r www.princeton.edu/~jzthree/datasets/ICML2014/ .
!rm -r www.princeton.edu/
!rm ICML2014/in*

/gdrive/Shareddrives/neurolusion/PhD/Secondary_Structure_Pred
 Dockerfile	  ICML2014   pytorch_examples.ipynb   src
 git_repo.ipynb   models    'Scheme SSpred.gslides'   SS_Pred_CNN.ipynb
--2021-01-19 15:44:37--  http://www.princeton.edu/~jzthree/datasets/ICML2014/
Resolving www.princeton.edu (www.princeton.edu)... 104.18.5.101, 104.18.4.101, 2606:4700::6812:565, ...
Connecting to www.princeton.edu (www.princeton.edu)|104.18.5.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2649 (2.6K) [text/html]
Saving to: ‘www.princeton.edu/~jzthree/datasets/ICML2014/index.html’

www.princeton.edu/~ 100%[===================>]   2.59K  --.-KB/s    in 0s      

2021-01-19 15:44:37 (312 MB/s) - ‘www.princeton.edu/~jzthree/datasets/ICML2014/index.html’ saved [2649/2649]

Loading robots.txt; please ignore errors.
--2021-01-19 15:44:37--  http://www.princeton.edu/robots.txt
Reusing existing connection to www.princeton.edu:80.
HTTP request sent, awaiting response... 301 Moved P

In [ ]:
# in case of being in my personal account
%cd /gdrive/Shareddrives/neurolusion/PhD/Secondary_Structure_Pred/protpred_2d/src

/gdrive/Shareddrives/neurolusion/PhD/Secondary_Structure_Pred/src


In [ ]:
import pdb.
import torch
import os

import numpy as np
from convolutional_nn import *
from one_hot_encoding import *
from datasetProcessing import *
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


if __name__ == "__main__":

  pdb.set_trace()
  path_preexist_model = '/gdrive/Shareddrives/neurolusion/PhD/Secondary_Structure_Pred/protpred_2d/models/ss_struct3.pth'
  dataset_path = "../ICML2014/cullpdb+profile_5926.npy.gz"
  n_prot = 5926; n_aa = 700; n_features = 57
  
  data_management = datasetProcessing(dataset_path, n_prot, n_aa, n_features)

  num_epochs = 1000
  total_samples = len(data_management.tr)
  batch_size = 256
  n_iterations = math.ceil(total_samples/batch_size)

  dataloader = DataLoader(dataset = data_management, batch_size = batch_size, num_workers = 4)


  # Beginning of training stage
  ss_struct_pred = convolutional_nn()

  '''
  # Find whether the GPU is still available
  CUDA=torch.cuda.is_available()
  if CUDA:
    ss_struct_pred = ss_struct_pred.cuda()
  '''

  # Definition of loss function
  loss_function = torch.nn.CrossEntropyLoss()
  
  # Selection of optimization algorithm to find the optimal weights in network with l2 norm regularization (weight decay)
  #optimizer=torch.optim.SGD(ss_struct_pred.parameters(), lr=0.01, weight_decay=1e-4)
  #optimizer=torch.optim.Adam(ss_struct_pred.parameters(), lr=0.01, weight_decay=1e-4)
  optimizer=torch.optim.SGD(ss_struct_pred.parameters(), lr=0.01, weight_decay=1e-4)


  training_iterations = 1000

  if os.path.isfile(path_preexist_model):
    print ("Loading Model")  
    state_dict = torch.load(path_preexist_model)
    #print(state_dict)
    ss_struct_pred.load_state_dict(state_dict)

  else:
    
    print ("Training Model")
    

    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(dataloader):

          optimizer.zero_grad()
          output = ss_struct_pred(inputs)
          loss = loss_function(output, torch.argmax(labels,dim=1))
          loss.backward()
          #print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
          #if (i+1) % 2 ==0:
          print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_iterations}, loss {loss.item()}')
          optimizer.step()
          torch.save(ss_struct_pred.state_dict(), path_preexist_model)
          torch.cuda.empty_cache()


> <ipython-input-3-f901e8a027d8>(16)<module>()
-> path_preexist_model = '/gdrive/Shareddrives/neurolusion/PhD/Secondary_Structure_Pred/models/ss_struct3.pth'
(Pdb) n
> <ipython-input-3-f901e8a027d8>(17)<module>()
-> dataset_path = "../ICML2014/cullpdb+profile_5926.npy.gz"
(Pdb) b 59
Breakpoint 1 at <ipython-input-3-f901e8a027d8>:59
(Pdb) 
Breakpoint 2 at <ipython-input-3-f901e8a027d8>:59
(Pdb) data_management.tt.shape
*** NameError: name 'data_management' is not defined
(Pdb) data_management.tt
*** NameError: name 'data_management' is not defined
(Pdb) l
 12  	
 13  	if __name__ == "__main__":
 14  	
 15  	  pdb.set_trace()
 16  	  path_preexist_model = '/gdrive/Shareddrives/neurolusion/PhD/Secondary_Structure_Pred/models/ss_struct3.pth'
 17  ->	  dataset_path = "../ICML2014/cullpdb+profile_5926.npy.gz"
 18  	  n_prot = 5926; n_aa = 700; n_features = 57
 19  	
 20  	  data_management = datasetProcessing(dataset_path, n_prot, n_aa, n_features)
 21  	
 22  	  num_epochs = 1000
(Pdb) b 59

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fe463ebeb70>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 124, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 47, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt: 


BdbQuit: ignored

In [ ]:
import pdb
import torch
import os

import numpy as np
from convolutional_nn import *
from one_hot_encoding import *
from datasetProcessing import *
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

'''
def Q8score_per_prot(t1,treal,hh):
  tpred=torch.Tensor(t1).type(torch.ByteTensor)
  #return (t1 == treal).mean()
  xxxx = (t1 == treal)
  mask = (treal ==8)

  mask = xxxx
  xxxx[mask]

'''
def Q8score_per_prot(t1,treal,hh):
  tpred=torch.Tensor(t1).type(torch.ByteTensor)
  cont=0
  for i in range(0,treal.shape[1]):
    chartpred= hh.get_inverted_encode(tpred[:,i].tolist())
    chartreal= hh.get_inverted_encode(treal[:,i].tolist())
    if chartpred == chartreal:
      if chartpred == 'NoSeq':
        continue
      else:
        cont+=1
    else:
      continue
  return float(cont)/float(treal.shape[1])
 


if __name__ == "__main__":
 
  path_preexist_model = '/gdrive/Shareddrives/neurolusion/PhD/Secondary_Structure_Pred/protpred_2d/models/ss_struct3.pth'
  dataset_path = "../ICML2014/cullpdb+profile_5926.npy.gz"
  n_prot = 5926; n_aa = 700; n_features = 57
  
  data_management = datasetProcessing(dataset_path, n_prot, n_aa, n_features)
  ss_struct_pred = convolutional_nn()

  if os.path.isfile(path_preexist_model):
    print ("Loading Model")  
    state_dict = torch.load(path_preexist_model)
    #print(state_dict)
    ss_struct_pred.load_state_dict(state_dict)

  pdb.set_trace()

  # calculation of Q8 accuracy
  average_Q8 = []
  output = ss_struct_pred(data_management.tt)
  ground_truth=data_management.ttl

  HSC_alphabet = ['L', 'B', 'E', 'G', 'I', 'H', 'S', 'T','NoSeq']
  HSC_onehot = one_hot_encoding(HSC_alphabet)
  
	# inverted = hh.get_inverted_encode(encode[0])
  for i in range(0,len(output)):
    get_output_sec = output[i]
    get_ground_truth_sec = ground_truth[i]
    average_Q8.append(Q8score_per_prot(get_output_sec, get_ground_truth_sec, HSC_onehot))

  Q8_acc = np.array(average_Q8).mean()  
  print(Q8_acc)


Loading Model
> <ipython-input-4-a489c13679da>(58)<module>()
-> average_Q8 = []
(Pdb) c
0.7530196078431373
